In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import zoopla_scrape as zs


to_scrape_df = pd.read_csv("to_scrape.csv")

to_scrape_df


#mcr_url = "https://www.zoopla.co.uk/for-sale/property/greater-manchester/?view_type=list&q=Greater%20Manchester&results_sort=newest_listings&search_source=home"
#london_url = "https://www.zoopla.co.uk/for-sale/property/london/?view_type=list&q=London&results_sort=newest_listings&search_source=home"



,location,link
0,manchester,https://www.zoopla.co.uk/for-sale/property/man...
1,salford,https://www.zoopla.co.uk/for-sale/property/gre...
2,bolton,https://www.zoopla.co.uk/for-sale/property/gre...
3,wigan,https://www.zoopla.co.uk/for-sale/property/wig...
4,stockport,https://www.zoopla.co.uk/for-sale/property/sto...
5,bury,https://www.zoopla.co.uk/for-sale/property/gre...
6,rochdale,https://www.zoopla.co.uk/for-sale/property/roc...
7,oldham,https://www.zoopla.co.uk/for-sale/property/old...
8,tameside,https://www.zoopla.co.uk/for-sale/property/tam...


In [12]:
df = pd.DataFrame()

In [13]:
data_frames = []

In [14]:
urls = to_scrape_df['link'].to_list()

for url in urls:
    data_frames.append(zs.get_house_data(url))
#data_frames.append(get_data(london_url))


Error count: 50


In [16]:
df = pd.concat(data_frames)

In [17]:
df

,title,address,price,station,beds,baths,receptions
0,3 bed terraced house for sale,"Richmond Street, Worsley Mesnes, Wigan WN3","£160,000",0.9 miles Wigan North Western,3,1,1
1,2 bed flat for sale,"The Boulevard, Didsbury, Manchester M20","£235,000",1.5 miles Burnage,2,2,1
2,2 bed flat for sale,"The Boulevard, West Didsbury, Manchester M20","£240,000",1.5 miles Burnage,2,2,1
3,5 bed detached house for sale,"Verdure Avenue, Heaton, Bolton BL1","£499,950",1 miles Lostock,5,2,2
4,3 bed terraced house for sale,"Yeoman's Close, Milnrow OL16","£180,000",1.2 miles Smithy Bridge,3,1,1
...,...,...,...,...,...,...,...
6020,1 bed flat for sale,"Nelson Square, Bolton, Lancashire BL1","£40,000",0.3 miles Bolton,1,1,0
6021,1 bed flat for sale,"Nelson Square, Bolton, Lancashire BL1","£40,000",0.3 miles Bolton,1,1,0
6022,1 bed flat for sale,"Nelson Square, Bolton, Lancashire BL1","£40,000",0.3 miles Bolton,1,1,0
6023,1 bed flat for sale,"Nelson Square, Bolton, Lancashire BL1","£40,000",0.3 miles Bolton,1,1,0


In [18]:
df.to_csv("zooplaiii-mcr.csv",index=False)

In [19]:
df

,title,address,price,station,beds,baths,receptions
0,3 bed terraced house for sale,"Richmond Street, Worsley Mesnes, Wigan WN3","£160,000",0.9 miles Wigan North Western,3,1,1
1,2 bed flat for sale,"The Boulevard, Didsbury, Manchester M20","£235,000",1.5 miles Burnage,2,2,1
2,2 bed flat for sale,"The Boulevard, West Didsbury, Manchester M20","£240,000",1.5 miles Burnage,2,2,1
3,5 bed detached house for sale,"Verdure Avenue, Heaton, Bolton BL1","£499,950",1 miles Lostock,5,2,2
4,3 bed terraced house for sale,"Yeoman's Close, Milnrow OL16","£180,000",1.2 miles Smithy Bridge,3,1,1
...,...,...,...,...,...,...,...
6020,1 bed flat for sale,"Nelson Square, Bolton, Lancashire BL1","£40,000",0.3 miles Bolton,1,1,0
6021,1 bed flat for sale,"Nelson Square, Bolton, Lancashire BL1","£40,000",0.3 miles Bolton,1,1,0
6022,1 bed flat for sale,"Nelson Square, Bolton, Lancashire BL1","£40,000",0.3 miles Bolton,1,1,0
6023,1 bed flat for sale,"Nelson Square, Bolton, Lancashire BL1","£40,000",0.3 miles Bolton,1,1,0
